## rechunk on laptop
#### Date: 27 March 2024

In [1]:
Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
from dask.distributed import Client, LocalCluster
# Create a LocalCluster that makes use of all the cores and memory on your machine
cluster = LocalCluster()

# Connect a client to the local cluster
client = Client(cluster)


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61415,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:61426,Total threads: 3
Dashboard: /proxy/8787/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:61418,


In [4]:
import xarray as xr
import zarr
import dask.array as dsa

##### work through https://rechunker.readthedocs.io/en/latest/tutorial.html#

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature")

In [ ]:
ds

In [ ]:
ds = ds.chunk({"time": 100})
ds.air.encoding = {}
ds.air.data

In [ ]:
! rm -rf air_temperature.zarr # clean up any existing temporary data
ds.to_zarr("air_temperature.zarr",consolidated=True)

In [ ]:
source = xr.open_zarr("air_temperature.zarr",consolidated=True)

In [ ]:
from rechunker import rechunk

target_chunks = {
    "air": {"time": 2920, "lat": 25, "lon": 1},
    "time": None,  # don't rechunk this array
    "lon": None,
    "lat": None,
}
max_mem = "1MB"

target_store = "group_rechunked.zarr"
temp_store = "group_rechunked-tmp.zarr"

# need to remove the existing stores or it won't work
!rm -rf group_rechunked.zarr group_rechunked-tmp.zarr
array_plan = rechunk(
    source, target_chunks, max_mem, target_store, temp_store=temp_store
)
array_plan

In [ ]:
array_plan.execute()

In [ ]:
xr.open_zarr("group_rechunked.zarr")

# ersstv5 example

In [18]:
ersstv5 = xr.tutorial.open_dataset('ersstv5')

In [19]:
ersstv5 = ersstv5.drop_vars('time_bnds')

In [20]:
ersstv5 = ersstv5.chunk({'lat':10,'lon':10,'time':10})

In [21]:
ersstv5

<xarray.Dataset> Size: 40MB
Dimensions:  (lat: 89, lon: 180, time: 624)
Coordinates:
  * lat      (lat) float32 356B 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 720B 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 5kB 1970-01-01 1970-02-01 ... 2021-12-01
Data variables:
    sst      (time, lat, lon) float32 40MB dask.array<chunksize=(10, 10, 10), meta=np.ndarray>
Attributes: (12/37)
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    description:               In situ data: ICOADS2.5 before 2007 and NCEP i...
    keywords_vocabulary:       NASA Global Change Master Directory (GCMD) Sci...
    keywords:                  Earth Science > Oceans > Ocean Temperature > S...
    instrument:                Conventional thermometers
    source_comment:            SSTs were observed by conventional thermometer...
    ...                        ...
    creator_url_original:      https://www.ncei.noaa.gov
    license:                   No constraints on data access or use
    comment:                   SSTs were observed by conventional thermometer...
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    dataset_title:             NOAA Extended Reconstructed SST V5
    data_modified:             2022-06-07

In [22]:
!rm -rf ersstv5.zarr
ersstv5.encoding={}
ersstv5.sst.encoding={}
!rm -rf ersstv5.zarr
ersstv5.to_zarr('ersstv5.zarr',consolidated=True)

In [23]:
source = xr.open_zarr('ersstv5.zarr',consolidated=True)

In [24]:
source

<xarray.Dataset> Size: 40MB
Dimensions:  (lat: 89, lon: 180, time: 624)
Coordinates:
  * lat      (lat) float32 356B 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 720B 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 5kB 1970-01-01 1970-02-01 ... 2021-12-01
Data variables:
    sst      (time, lat, lon) float32 40MB dask.array<chunksize=(10, 10, 10), meta=np.ndarray>
Attributes: (12/37)
    Conventions:               CF-1.6, ACDD-1.3
    References:                https://www.ncdc.noaa.gov/data-access/marineoc...
    cdm_data_type:             Grid
    citation:                  Huang et al, 2017: Extended Reconstructed Sea ...
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    comment:                   SSTs were observed by conventional thermometer...
    ...                        ...
    project:                   NOAA Extended Reconstructed Sea Surface Temper...
    source:                    In situ data: ICOADS R3.0 before 2015, NCEP in...
    source_comment:            SSTs were observed by conventional thermometer...
    standard_name_vocabulary:  CF Standard Name Table (v40, 25 January 2017)
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    title:                     NOAA ERSSTv5 (in situ only)

In [27]:
from rechunker import rechunk

target_chunks = {"time": 100, "lat": 10, "lon": 100}
max_mem = "1MB"

target_store = "target_rechunked.zarr"
temp_store = "rechunking-tmp.zarr"

# need to remove the existing stores or it won't work
!rm -rf rechunking-tmp.zarr target_rechunked.zarr
array_plan = rechunk(
    source, target_chunks, max_mem, target_store, temp_store=temp_store
)
array_plan

<Rechunked>
* Source      : <xarray.Dataset> Size: 40MB
Dimensions:  (lat: 89, lon: 180, time: 624)
Coordinates:
  * lat      (lat) float32 356B 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 720B 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 5kB 1970-01-01 1970-02-01 ... 2021-12-01
Data variables:
    sst      (time, lat, lon) float32 40MB dask.array<chunksize=(10, 10, 10), meta=np.ndarray>
Attributes: (12/37)
    Conventions:               CF-1.6, ACDD-1.3
    References:                https://www.ncdc.noaa.gov/data-access/marineoc...
    cdm_data_type:             Grid
    citation:                  Huang et al, 2017: Extended Reconstructed Sea ...
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    comment:                   SSTs were observed by conventional thermometer...
    ...                        ...
    project:                   NOAA Extended Reconstructed Sea Surface Temper...
    source:                    In situ data: ICOADS R3.0 before 2015, NCEP in...
    source_comment:            SSTs were observed by conventional thermometer...
    standard_name_vocabulary:  CF Standard Name Table (v40, 25 January 2017)
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    title:                     NOAA ERSSTv5 (in situ only)

* Intermediate: <zarr.hierarchy.Group '/'>

* Target      : <zarr.hierarchy.Group '/'>

In [28]:
array_plan.execute()

2024-03-27 14:57:57,945 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-03-27 14:57:58,075 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-03-27 14:57:58,522 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-03-27 14:57:58,656 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-03-27 14:57:59,119 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-03-27 14:57:59,262 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2024-03-27 14:57:59,717 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-03-27 14:57:59,855 - distributed.utils_perf - WARNING - full garbage collections took

<zarr.hierarchy.Group '/'>

In [29]:
xr.open_zarr('target_rechunked.zarr')

/var/folders/2t/fb36j9595b5_0sczyy0ry7rw0000gp/T/ipykernel_95879/2949659203.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  xr.open_zarr('target_rechunked.zarr')


<xarray.Dataset> Size: 40MB
Dimensions:  (lat: 89, lon: 180, time: 624)
Coordinates:
  * lat      (lat) float32 356B 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 720B 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 5kB 1970-01-01 1970-02-01 ... 2021-12-01
Data variables:
    sst      (time, lat, lon) float32 40MB dask.array<chunksize=(100, 10, 100), meta=np.ndarray>
Attributes: (12/37)
    Conventions:               CF-1.6, ACDD-1.3
    References:                https://www.ncdc.noaa.gov/data-access/marineoc...
    cdm_data_type:             Grid
    citation:                  Huang et al, 2017: Extended Reconstructed Sea ...
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    comment:                   SSTs were observed by conventional thermometer...
    ...                        ...
    project:                   NOAA Extended Reconstructed Sea Surface Temper...
    source:                    In situ data: ICOADS R3.0 before 2015, NCEP in...
    source_comment:            SSTs were observed by conventional thermometer...
    standard_name_vocabulary:  CF Standard Name Table (v40, 25 January 2017)
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    title:                     NOAA ERSSTv5 (in situ only)